### Imports


In [21]:
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, T5ForConditionalGeneration, T5Tokenizer
import nltk
import string
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
import multiprocessing
from functools import partial
import concurrent.futures
import pandas as pd

### Load dataset + data preprocessing

In [22]:
df=pd.read_csv('datasets/US_stocks.csv')
labels = df['Sector'].unique()
df=df.head(n=100)
sentences = df['Description']
labels.shape

(11,)

# Tokenize + Punctuation removal + Lemmatize

In [23]:
def tokenizer(sentence):
    lemmatizer=WordNetLemmatizer()
    tokens = nltk.word_tokenize(sentence.lower())
    stop_words = stopwords.words('english')
    words = [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in tokens if word not in string.punctuation]
    sentence=' '.join(words)
    return sentence

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def text_summarization(input_text, max_length=150):
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    tokenizer = T5Tokenizer.from_pretrained("t5-small")

    input_text = "summarize: " + input_text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = model.generate(input_ids, max_length=max_length, num_beams=2, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

def process_sentence(sentence):
    preprocessed_sentence = tokenizer(sentence)
    summary = text_summarization(preprocessed_sentence, 180)
    return summary

with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    sentences = list(executor.map(process_sentence, sentences))

#sentences=[text_summarization(tokenizer(sentence),180) for sentence in sentences]


# mDeBERTa with CUDA support for faster compute times

In [24]:
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=True,batched=True, numprocs=8)
classifier = transformers.pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli", device=0)#, tokenizer=tokenizer) 
prediction = [classifier(sentence, labels, multi_label=True) for sentence in sentences]

C:\Users\hugor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
C:\Users\hugor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\transformers\pipelines\base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


### Metrics

In [25]:
predicted_label=[prediction[i]['labels'][0] for i in range(df.shape[0])]
df['Predicted Label'] = predicted_label

accuracy = accuracy_score(df['Sector'], df['Predicted Label'])
precision = precision_score(df['Sector'], df['Predicted Label'],average='weighted')
recall = recall_score(df['Sector'], df['Predicted Label'],average='weighted')
f1 = f1_score(df['Sector'], df['Predicted Label'],average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.81
Precision: 0.9286666666666668
Recall: 0.81
F1 Score: 0.8592307692307692


C:\Users\hugor\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
